<a href="https://colab.research.google.com/github/dongchanlim/CS450/blob/master/04_Prove_Assignment_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import files

file = files.upload()

Saving crx.data to crx.data


In [149]:
credit = pd.read_csv("crx.data", header = None, na_values = ["?"], skipinitialspace= True)
# set column name from A1 to A16
credit.columns = ["A" + str(i) for i in range(1,17)]
credit

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [150]:
credit.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [151]:
credit.isna().sum(axis = 0)

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [152]:
from sklearn.impute import SimpleImputer

imp_numeric = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_discrete = SimpleImputer(missing_values= np.nan, strategy= 'most_frequent')

credit[["A2", "A14"]] = imp_numeric.fit_transform(credit[["A2", "A14"]])
credit[["A1", "A4", "A5", "A6", "A7"]] = imp_discrete.fit_transform(credit[["A1", "A4", "A5", "A6", "A7"]])

credit.isna().sum(axis = 0)

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

In [0]:
from sklearn.model_selection import train_test_split

X = credit.drop(columns = "A16", axis = 1)
y = credit["A16"].map({"+":0, "-":1})

X = pd.get_dummies(X, columns = X.select_dtypes("object").columns)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

In [159]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion = "entropy", random_state = 0)
score = cross_val_score(tree, X = X, y = y, cv = 5)

print("Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Accuracy: 0.80 (+/- 0.14)


In [160]:
from sklearn.metrics import accuracy_score

tree.fit(X_train, y_train)
predicted_y = tree.predict(X_test)

accuracy_score(y_test, predicted_y)


0.7777777777777778